#Projet introduction à l'analyse d'image

In [132]:
!pip install pillow-heif
from google.colab import drive
import os
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pillow_heif
import csv
import math
from functools import cmp_to_key
from google.colab.patches import cv2_imshow


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Fonctions pour le prétraitement des images

In [133]:
#https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
def image_resize(image, width = None, height = None, inter = cv.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image

    resized = cv.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

############################################################################################################

"""
kernel = np.array([[0,1,0,1,0],
                   [1,1,1,1,1],
                   [0,1,0,1,0],
                   [1,1,1,1,1],
                   [0,1,0,1,0]])
"""
def dilation(image,kernel):
    imageF = np.zeros_like(image)
    res = image.copy();
    for x in range(res.shape[0]):
        for y in range(res.shape[1]):
            xmin= max(0,x-(kernel.shape[0]//2))
            xmax= min(res.shape[0]-1,x+(kernel.shape[0]//2))
            ymin= max(0,y-(kernel.shape[1]//2))
            ymax= min(res.shape[1]-1,y+(kernel.shape[1]//2))
            voisins = image[xmin:xmax+1,ymin:ymax+1]
            imageF[x,y]=np.max(voisins)
    return imageF;


def erosion(image,kernel):
    imageF = np.zeros_like(image)
    res = image.copy();
    for x in range(res.shape[0]):
        for y in range(res.shape[1]):
            xmin= max(0,x-(kernel.shape[0]//2))
            xmax= min(res.shape[0]-1,x+(kernel.shape[0]//2))
            ymin= max(0,y-(kernel.shape[1]//2))
            ymax= min(res.shape[1]-1,y+(kernel.shape[1]//2))
            voisins = image[xmin:xmax+1,ymin:ymax+1]
            imageF[x,y]=np.min(voisins)
    return imageF;


def opening(image,kernel):
    res = image.copy();
    eroded = erosion(res,kernel)
    opened = dilation(eroded,kernel)
    return opened

def closing(image,kernel):
    res = image.copy();
    dilated = dilation(res,kernel)
    closed = erosion(dilated,kernel)
    return closed

############################################################################################################

def localized_otsu(image):
    # Convert image to grayscale
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Calculate block size based on image dimensions
    block_size = max(3, int(min(image.shape[:2]) / 10))  # Adcoin_countust the divisor for desired block size
    block_size = block_size if block_size % 2 == 1 else block_size + 1  # Ensure blockSize is odd

    # Calculate mean and standard deviation for each block
    mean = cv.blur(gray, (block_size, block_size))
    mean_square = cv.blur(gray ** 2, (block_size, block_size))
    variance = mean_square - mean ** 2

    # Calculate minimum and maximum thresholds based on mean and standard deviation
    k = 0.4  # Adcoin_countust this value for desired sensitivity
    min_threshold = mean - k * np.sqrt(variance)
    max_threshold = mean + k * np.sqrt(variance)

    # Apply thresholding
    localized_otsu = cv.inRange(gray, min_threshold.astype(np.uint8), max_threshold.astype(np.uint8))

    return localized_otsu

############################################################################################################

def extract_ROIs(circles, image):
    ROIs = []

    for i in circles[0, :]:
        # Prepare a black canvas:
        height, width = image.shape[:2]
        canvas = np.zeros((height, width))

        # Draw the outer circle:
        color = (255, 255, 255)
        thickness = -1
        centerX = i[0]
        centerY = i[1]
        radius = i[2]
        cv.circle(canvas, (centerX, centerY), radius, color, thickness)

        # Create a copy of the input and mask input:
        image_copy = image.copy()
        image_copy[canvas == 0] = (0, 0, 0)

        # Crop the roi:
        x = centerX - radius
        y = centerY - radius
        h = 2 * radius
        w = 2 * radius

        cropped_img = image_copy[y - 3:y + h + 3, x - 3:x + w + 3]

        # Store the ROI:
        ROIs.append(cropped_img)
    return ROIs

############################################################################################################

COPPER_CENT_BGR = (30., 100., 160.)
BRASS_CENT_BGR = (70., 120., 120.)

COPPER_CENT_VALUE = (0.01 + 0.02 + 0.05) / 3.
BRASS_CENT_VALUE = (0.10 + 0.20 + 0.50) / 3.

def get_mean_HSV(image, mask):
    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    return cv.mean(image_hsv, mask=mask)

def is_euro(image, mask):
    hsv = get_mean_HSV(image, mask)
    return hsv[0] >= 14 and hsv[0] < 18 and hsv[1] > 50 and hsv[1] <=165 and hsv[2] > 80 and hsv[2] <=210

def is_copper_cent(image, mask):
    # Convert the mask to the correct type if needed
    mean_color = cv.mean(image, mask=mask)[:3]
    #print("mean color norm: ", cv.norm(mean_color, COPPER_CENT_BGR))
    #print("mean color, copper color:", mean_color, COPPER_CENT_BGR)
    #return cv.norm(mean_color, COPPER_CENT_BGR) < cv.norm(mean_color, BRASS_CENT_BGR)
    prc_GR = mean_color[1] / mean_color[2]
    prc_BR = mean_color[0] / mean_color[2]
    return prc_GR > prc_BR and \
           (prc_GR - prc_BR) < .35


def is_brass_cent(image, mask):
    # Convert the mask to the correct type if needed
    mean_color = cv.mean(image, mask=mask)[:3]
    #print("mean color norm: ", cv.norm(mean_color, BRASS_CENT_BGR))
    #print("mean color, brass color:", mean_color, BRASS_CENT_BGR)
    #return cv.norm(mean_color, BRASS_CENT_BGR) < cv.norm(mean_color, COPPER_CENT_BGR)
    diff_RG = mean_color[2] - mean_color[1]
    diff_GB = mean_color[1] - mean_color[0]
    return diff_GB > diff_RG * 2.3

def get_cent_value(image):
    # Create a mask for the circular region
    coin_mask = np.zeros(image.shape[:2], np.uint8)
    center = (49, 49)  # Using tuple for center coordinates
    radius = 42
    cv.circle(coin_mask, center, radius, 255, -1)

    if is_brass_cent(image, coin_mask):
        #print("BRASS:", BRASS_CENT_VALUE)
        return BRASS_CENT_VALUE
    elif is_copper_cent(image, coin_mask):
        #print("COPPER:", COPPER_CENT_VALUE)
        return COPPER_CENT_VALUE
    else:
        #print("NOT A COIN:", 0.00)
        return 0.00


############################################################################################################

def morphology_fill(image, kernel):
    thresh = opening(image, kernel)

    morph = cv.morphologyEx(image, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 15)))
    # Mask used to flood filling.
    # NOTE: the size needs to be 2 pixels bigger on each side than the input image
    h, w = image.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    im_floodfill = image.copy()

    # Floodfill from point (0, 0)
    cv.floodFill(im_floodfill, mask, (15,15), 255)

    # Invert floodfilled image
    im_floodfill_inv = cv.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground
    im_out = image | im_floodfill_inv

    return im_out

############################################################################################################

def is_protruding(center, radius, image):
    height, width = image.shape[:2]
    height -= 1
    width -= 1

    center_x, center_y = center[:2]
    return (center_y + radius >= height) or \
           (center_y - radius <= 0) or \
           (center_x + radius >= width) or \
           (center_x - radius <= 0)

def fix_circle_size(circle, center, radius, image):
    new_radius = 100000

    height, width = image.shape[:2]
    height -= 1
    width -= 1

    center_x, center_y = center[:2]

    #print("center:", center)
    #print("radius:", radius)
    #print("height:", height)
    #print("width:", width)
    if center_y + radius >= height:
        temp_radius = radius - (center_y + radius - height - 5)
        #print(temp_radius)
        if temp_radius < new_radius: new_radius = temp_radius

    if center_y - radius <= 0:
        temp_radius = radius - (radius - center_y + 5)
        #print(temp_radius)
        if temp_radius < new_radius: new_radius = temp_radius

    if center_x + radius >= width:
        temp_radius = radius - (center_x + radius - width - 5)
        if temp_radius < new_radius: new_radius = temp_radius

    if center_x - radius <= 0:
        temp_radius = radius - (radius - center_x + 5)
        if temp_radius < new_radius: new_radius = temp_radius

    if new_radius < radius:
        circle[2] = new_radius

    return circle

## Fonction pour lire les images

In [134]:
def prepare_image(image_name, folder_name):
    image = cv.imread(f"{folder_name}/{image_name}")

    # Try opening the image
    if ".HEIC" in image_name:
        if os.path.exists(f"{folder_name}/{image_name}.png"):
            try:
                os.remove(f"{folder_name}/{image_name}")
            except:
                image = cv.imread(f"{folder_name}/{image_name}.png")
        else:
            try:
                heif_file = pillow_heif.open_heif(f"{folder_name}/{image_name}", convert_hdr_to_8bit=False, bgr_mode=True)
                np_array = np.asarray(heif_file)
                cv.imwrite(f"{folder_name}/{image_name}.png", np_array)
                image = cv.imread(f"{folder_name}/{image_name}.png")
            except:
                print('Could not open or find the image named:', image_name)
        print("Successfully opened the following .heic image:", image_name)
    elif image is not None:
        print("Successfully opened the following image:", image_name)
    elif image is None:
        print('Could not open or find the image named:', image_name)
        return

    return image



## Importation des images et de leurs info
(à changer en fonction de l'emplacement dans le drive)

In [135]:
import pandas as pd

folder_name = "data/"

df = pd.read_csv("data_annotations.csv")
df

,image_name,nb_piece,value
0,006.jpeg,23,3 euros 32 centimes
1,007.jpeg,23,3 euros 32 centimes
2,008.jpeg,14,1 euro 66 centimes
3,009.jpeg,14,1 euro 66 centimes
4,010.jpeg,10,1 euro 19 centimes
...,...,...,...
70,PXL_20240206_141524308.jpg,1,1 euro
71,PXL_20240206_141535905.jpg,3,3 euros 20 centimes
72,PXL_20240206_141545421.jpg,6,4 euros 80 centimes
73,PXL_20240206_231416015.jpg,4,35 centimes


## Fonction pour la détection du nombre de pieces dans les images

In [194]:

def detect_circle(image):

  kernel = np.array([[0,0,1,0,0],
                    [0,1,1,1,0],
                    [1,1,0,1,1],
                    [0,1,1,1,0],
                    [0,0,1,0,0]])
  #kernel = np.ones((5, 5), np.uint8)

  SCALE_RATIO = 1

  new_width = round(300*SCALE_RATIO)
  new_height = round(400*SCALE_RATIO)

  image = image_resize(image, width = new_width, height = new_height)
  #cv2_imshow(image)

  dummy_image = image.copy()

  gray_image = cv.cvtColor(dummy_image, cv.COLOR_BGR2GRAY)

  blur_image = cv.bilateralFilter(gray_image, 5, 150, 150)
  #blur_image = cv.GaussianBlur(gray_image, (3,3), 0)
  #blur_image = cv.medianBlur(gray_image, 5)
  ##cv2_imshow(blur_image)

  # Find contours
  canny_image = cv.Canny(blur_image,75,200)
  canny_image_blur = cv.GaussianBlur(canny_image, (3,3), 0)
  contours, _ = cv.findContours(canny_image_blur, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

  # The coin detection method is by default the one using findContours
  are_contours_coherent = True

  # Filtering contours of circular inertia
  contour_list = []
  for contour in contours:
      approx = cv.approxPolyDP(contour,0.005*cv.arcLength(contour,True),True)
      area = cv.contourArea(contour)
      #print("area:", area)

      # Filter out noise - If an area is absurdly large, abandon the findContours method
      if (len(approx) > 8) & (30 < area < 15000*SCALE_RATIO):
          contour_list.append(contour)
      elif 15000*SCALE_RATIO < area:
          are_contours_coherent = False
          break

  # Iterate a second time over the coin detection process if the findContours method is not productive (0 coins detected)
  for iteration in range(2):
      if are_contours_coherent:
          image_with_contours = np.zeros_like(blur_image)
          cv.drawContours(image_with_contours, contour_list,  -1, (255,0,0), 2)
          image_for_hough_transform = closing(image_with_contours, kernel)
          #image_for_hough_transform = morphology_fill(image_with_contours, kernel)
          #cv2_imshow(canny_image)
          #cv2_imshow(canny_image_blur)
      else:
          image_for_hough_transform = canny_image

      #cv2_imshow(image_for_hough_transform)

      # Hough Transform function
      minDist = round(15*SCALE_RATIO)
      minRadius = round(10*SCALE_RATIO)
      maxRadius = round(150*SCALE_RATIO)

      circles = cv.HoughCircles(image_for_hough_transform, cv.HOUGH_GRADIENT, 1, \
                                minDist = minDist, \
                                param1 = 120, param2 = 20, \
                                minRadius = minRadius, maxRadius = maxRadius)
      # minDist: Minimum distance between the centers of the detected circles.
      # param1: The larger of the two thresholds passed to the Canny edge detector.
      # param2: Threshold for the minimum number of votes required for a circle's center to be detected. A smaller value may lead to false detections.
      # minRadius: The minimum radius of the circles to detect.
      # maxRadius: The maximum radius of the circles to detect.

      coin_count = 0

      #検出された際に動くようにする。
      if circles is not None and len(circles) > 0:

          #型をfloat32からunit16に変更。
          circles = np.uint16(np.around(circles))

          if are_contours_coherent:
              contour_list = sorted(contour_list, key=lambda contour: cv.contourArea(contour))
              median_contour = contour_list[len(contour_list)//2]
              median_radius = math.sqrt(cv.contourArea(median_contour)/math.pi)

              upper_radius_limit = round(median_radius * 1.5)
              lower_radius_limit = round(median_radius * .6)

              lower_distance_limit = lower_radius_limit * 2
          else:
              sorted_radiuses = sorted(circles[0,:,2])
              median_radius = sorted_radiuses[len(sorted_radiuses)//2]
              mean_radius = round(np.mean(sorted_radiuses))
              #print("mean radius:", mean_radius)
              #print("median radius:", median_radius)

              upper_radius_limit = round(median_radius * 1.4)
              #upper_radius_limit = round(mean_radius * 1.6)

              #lower_radius_limit = round(median_radius * .8)
              lower_radius_limit = sorted_radiuses[0]
              lower_distance_limit = round(median_radius * .4)*2
              #print("lower distance limit:", lower_distance_limit)

          circles = cv.HoughCircles(image_for_hough_transform, cv.HOUGH_GRADIENT, 1, \
                                    minDist = lower_distance_limit, \
                                    param1 = 170, param2 = 20, \
                                    minRadius = lower_radius_limit, maxRadius = upper_radius_limit)

          # Iterate over detected circles
          if circles is not None and len(circles) > 0:
              circles = np.uint16(np.around(circles))
              circle_list = circles[0,:]
              for circle_index in range(len(circle_list)):
                  circle = circle_list[circle_index]
                  # Extract circle parameters
                  center = (circle[0], circle[1])  # Center coordinates
                  radius = circle[2]  # Radius
                  if is_protruding(center, radius, image):
                      circles[0, circle_index] = fix_circle_size(circle, center, radius, image)

                  # Draw the outer circle
                  cv.circle(dummy_image, center, radius, (0, 191, 255), 2)
                  # Draw the center
                  cv.circle(dummy_image, center, 2, (255, 255, 0), 2)

                  # Increment circle count
                  coin_count += 1

              break

          elif circles is not None and not are_contours_coherent:
              break

      are_contours_coherent = False


  #print(circles[0,:])
  # Draw the total count on the image
  fontType = cv.FONT_HERSHEY_DUPLEX
  cv.putText(dummy_image, f'{str(coin_count)} coins', (30,30), fontType, \
            1, (255, 255, 0), 1, cv.LINE_AA)
  #cv2_imshow(dummy_image)
  return circles, coin_count



## Fonction pour la detection de valeur pour chaque pieces détecté

In [189]:

def detect_value(circles, image):

  SCALE_RATIO = 1

  new_width = round(300*SCALE_RATIO)
  new_height = round(400*SCALE_RATIO)

  image = image_resize(image, width = new_width, height = new_height)

  total_value = 0.00
  #cv2_imshow(image)
  ROIs = extract_ROIs(circles, image)
  for ROI in ROIs:
      coin_value = 0.00
      #print("circle count:", len(ROIs))
      circles = None

      ROI = image_resize(ROI, width = 100, height = 100)
      #cv2_imshow(ROI)

      ROI_copy = ROI.copy()

      ROI_blur = cv.bilateralFilter(ROI_copy, 3, 200, 200)

      ROI_gray = cv.cvtColor(ROI_blur, cv.COLOR_BGR2GRAY)

      #ROI_blur = cv.GaussianBlur(ROI_gray, (3,3), 0)
      #cv2_imshow(ROI_gray)

      ROI_gray = cv.bitwise_not(ROI_gray)
      ROI_thresh = cv.adaptiveThreshold(ROI_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV,23,2)

      #ROI_blur = cv.GaussianBlur(ROI_gray, (5,5), 0)
      #cv2_imshow(ROI_thresh)


      circles = cv.HoughCircles(ROI_thresh, cv.HOUGH_GRADIENT, 1, 1, param1 = 120, param2 = 1, minRadius = 25, maxRadius = 35)
      # param1 : canny()エッジ検出器に渡される2つの閾値のうち、大きいほうの閾値0
      # param2 : 円の中心を検出する際の投票数の閾値。小さくなるほど、誤検出が起こる可能性がある。
      # minRadius : 検出する円の最小値
      # maxRadius : 検出する円の最大値

      #検出された際に動くようにする。
      if circles is not None and len(circles) > 0:
          ##print(circles)
          #型をfloat32からunit16に変更。
          circles = np.uint16(np.around(circles))
          center_point = np.array([49, 49], dtype=np.uint16)
          points = circles[0,:]
          ##print(center_point)
          ##print(points)
          points = np.array(sorted(points, key=lambda item: cv.norm(np.int8(item[:2]), np.int8(center_point[:2]))))
          #print(points)
          if cv.norm(points[0,:2], center_point) > 9:
              #print("norm:", cv.norm(points[0,:2], center_point))
              #print("norm:", cv.norm(points[1,:2], center_point))
              #print("not a euro coin")
              total_value += get_cent_value(ROI)
              continue

          inner_circle = points[0]

          # 外側の円を描く
          cv.circle(ROI_copy,(inner_circle[0], inner_circle[1]), inner_circle[2], (0, 191, 255), 2)
          # 中心の円を描く
          cv.circle(ROI_copy,(inner_circle[0], inner_circle[1]), 2, (255, 255, 0), 2)

          #円の合計数を表示
          #cv2_imshow(ROI_copy)

          inner_center = np.array(inner_circle[:2])
          inner_radius = inner_circle[2]

          # Calculate area of the circle
          inner_area = np.pi * inner_radius ** 2
          #print(f"Area of the circle: {inner_area}, Radius: {inner_radius}")

          # Create a mask for the circular region
          inner_mask = np.zeros(ROI_copy.shape[:2], np.uint8)
          cv.circle(inner_mask, inner_center, inner_radius, 255, -1)

          # Calculate the mean color within the circular region
          ROI_copy = ROI.copy()
          inner_mean_color = cv.mean(ROI_copy, mask=inner_mask)

          #print("Mean Color (BGR) inside the circle:", inner_mean_color)

          circles = cv.HoughCircles(ROI_thresh, cv.HOUGH_GRADIENT, 1, 5, param1 = 130, param2 = 1, minRadius = 35, maxRadius = 60)

          if circles is not None and len(circles) > 0:

              points = circles[0,:]
              #型をfloat32からunit16に変更。
              points = np.uint16(np.around(points))
              ##print(center_point)
              ##print(points)

              points = sorted(points, key=lambda item: cv.norm(np.int8(item[:2]), np.int8(center_point[:2])))
              #print(points)

              for outer_circle in points:
                  outer_circle[2] -= 3

                  if cv.norm(outer_circle[:2], center_point) > 10:
                      #print("not a euro coin")
                      break

                  ROI_copy = ROI.copy()

                  # 外側の円を描く
                  cv.circle(ROI_copy,(outer_circle[0], outer_circle[1]), outer_circle[2], (0, 191, 255), 2)
                  # 中心の円を描く
                  cv.circle(ROI_copy,(outer_circle[0], outer_circle[1]), 2, (255, 255, 0), 2)

                  #円の合計数を表示
                  #cv2_imshow(ROI_copy)

                  outer_center = np.array(outer_circle[:2])
                  ##print(outer_center)
                  if cv.norm(inner_center, outer_center) > 5:
                      continue

                  outer_radius = outer_circle[2]
                  if outer_radius < inner_radius * 1.3:
                      continue

                  # Calculate area of the circle
                  outer_area = (np.pi * outer_radius ** 2) - inner_area
                  #print(f"Area of the circle: {outer_area}, Radius: {outer_radius}")

                  # Create a mask for the circular region
                  outer_mask = np.zeros(ROI_copy.shape[:2], np.uint8)
                  cv.circle(outer_mask, outer_center, outer_radius, 255, -1)
                  cv.circle(outer_mask, outer_center, inner_radius, 0, -1)

                  ROI_copy = ROI.copy()
                  res = cv.bitwise_and(ROI_copy,ROI_copy,mask = outer_mask)
                  #cv2_imshow(res)

                  # Calculate the mean color within the circular region
                  outer_mean_color = cv.mean(ROI_copy, mask=outer_mask)

                  #print("Mean Color (BGR) inside the circle:", outer_mean_color)
                  #print("Norm between the two mean colors:",cv.norm(inner_mean_color, outer_mean_color))

                  if (((inner_mean_color[1] - inner_mean_color[0]) + \
                      (inner_mean_color[2] - inner_mean_color[0])) > \
                      ((outer_mean_color[1] - outer_mean_color[0]) + \
                      (outer_mean_color[2] - outer_mean_color[0]))) and \
                      cv.norm(inner_mean_color, outer_mean_color) > 19.3:

                      coin_value = 2.00
                      #print("2 euros")
                      break
                  elif cv.norm(inner_mean_color, outer_mean_color) > 19.3:
                      coin_value = 1.00
                      #print("1 euro")
                      break
                  else:
                      #print("not a euro coin")
                      break

              if coin_value == 0.00:
                  coin_value = get_cent_value(ROI)
          else:
              coin_value = get_cent_value(ROI)
      else:
          coin_value = get_cent_value(ROI)
      total_value += coin_value
      #print(f"total value: {total_value} euros")
  #print(f"grand total: {round(total_value, 2)} euros")
  return total_value


### Test pour une image

In [191]:
image = prepare_image("8.jpg", folder_name)

print("ETAPE 1")
circles, coins = detect_circle(image)
print(f"nombre de pieces detecté :{coins}")
print("ETAPE 2")
value = detect_value(circles, image)
print(f"valeur total : {value}")

Successfully opened the following image: 8.jpg
ETAPE 1
nombre de pieces detecté :3
ETAPE 2
valeur total : 1.0533333333333332


## Fonction pour obtenir la valeur des pieces d'une image grace au csv

In [127]:
def convert_value_to_numeric(value_str):
    words = value_str.split()
    euros = 0
    centimes = 0

    for i in range(len(words)):
        if words[i] == 'euros' or words[i] == 'euro':
            euros = int(words[i-1]) if i > 0 else 0
        elif words[i] == 'centimes' or words[i] == 'centime':
            centimes = int(words[i-1]) if i > 0 else 0

    total_value = euros + centimes / 100
    return total_value

## Calcul de la MAE pour les images detectées avec le bon nombre de pieces

In [192]:
from sklearn.metrics import mean_absolute_error

real_values = []
predicted_values = []

for index, row in df.iterrows():
    filename = row['image_name']
    num_real_pieces = row['nb_piece']
    num_real_value = row['value']
    num_real_value_numeric = convert_value_to_numeric(num_real_value)

    print(f"IMAGE {filename} :")

    image = prepare_image(filename, folder_name)
    cirles, coins = detect_circle(image)
    print(f"cercles detecté : {coins}")
    print(f"nb piece reel : {num_real_pieces}")
    if(coins==num_real_pieces):
      value = detect_value(circles, image)
      print(f"valeur detecté : {value}")
      print(f"valeur reel : {num_real_value_numeric}\n")
      real_values.append(num_real_value_numeric)
      predicted_values.append(value)
    else:
      print("Mauvais compte de piece")

mae = mean_absolute_error(real_values, predicted_values)
print(f"Mean Absolute Error : {mae}")

IMAGE 006.jpeg :
Successfully opened the following image: 006.jpeg
cercles detecté : 3
nb piece reel : 23
Mauvais compte de piece
IMAGE 007.jpeg :
Successfully opened the following image: 007.jpeg
cercles detecté : 10
nb piece reel : 23
Mauvais compte de piece
IMAGE 008.jpeg :
Successfully opened the following image: 008.jpeg
cercles detecté : 0
nb piece reel : 14
Mauvais compte de piece
IMAGE 009.jpeg :
Successfully opened the following image: 009.jpeg
cercles detecté : 1
nb piece reel : 14
Mauvais compte de piece
IMAGE 010.jpeg :
Successfully opened the following image: 010.jpeg
cercles detecté : 3
nb piece reel : 10
Mauvais compte de piece
IMAGE 011.jpeg :
Successfully opened the following image: 011.jpeg
cercles detecté : 5
nb piece reel : 5
valeur detecté : 0.08
valeur reel : 0.65

IMAGE 012.jpeg :
Successfully opened the following image: 012.jpeg
cercles detecté : 5
nb piece reel : 5
valeur detecté : 0.08
valeur reel : 0.54

IMAGE 013.jpeg :
Successfully opened the following imag

## Calcul du pourcentage d'images détectées avec le bon nombre de pieces

In [195]:
total_images = 0
correct_images = 0

for index, row in df.iterrows():
    filename = row['image_name']
    num_real_pieces = row['nb_piece']
    num_real_value = row['value']

    total_images += 1

    print(f"IMAGE {filename} :")

    image = prepare_image(filename, folder_name)
    circles, coins = detect_circle(image)
    print(f"cercles détectés : {coins}")
    print(f"nb pièces réelles : {num_real_pieces}")

    if coins == num_real_pieces:
        correct_images += 1

percentage_correct = (correct_images / total_images) * 100
print(f"Pourcentage d'images avec le bon nombre de pièces : {percentage_correct}%")


IMAGE 006.jpeg :
Successfully opened the following image: 006.jpeg
cercles détectés : 21
nb pièces réelles : 23
IMAGE 007.jpeg :
Successfully opened the following image: 007.jpeg
cercles détectés : 19
nb pièces réelles : 23
IMAGE 008.jpeg :
Successfully opened the following image: 008.jpeg


<ipython-input-133-d872bd4b9a4e>:231: RuntimeWarning: overflow encountered in scalar subtract
  (center_y - radius <= 0) or \


cercles détectés : 29
nb pièces réelles : 14
IMAGE 009.jpeg :
Successfully opened the following image: 009.jpeg
cercles détectés : 13
nb pièces réelles : 14
IMAGE 010.jpeg :
Successfully opened the following image: 010.jpeg
cercles détectés : 9
nb pièces réelles : 10
IMAGE 011.jpeg :
Successfully opened the following image: 011.jpeg
cercles détectés : 5
nb pièces réelles : 5
IMAGE 012.jpeg :
Successfully opened the following image: 012.jpeg
cercles détectés : 5
nb pièces réelles : 5
IMAGE 013.jpeg :
Successfully opened the following image: 013.jpeg
cercles détectés : 4
nb pièces réelles : 4
IMAGE 014.jpeg :
Successfully opened the following image: 014.jpeg
cercles détectés : 14
nb pièces réelles : 14
IMAGE 015.jpeg :
Successfully opened the following image: 015.jpeg
cercles détectés : 9
nb pièces réelles : 9
IMAGE 0e01c.jpg :
Successfully opened the following image: 0e01c.jpg
cercles détectés : 10
nb pièces réelles : 1
IMAGE 0e10c.jpg :
Successfully opened the following image: 0e10c.jp

<ipython-input-133-d872bd4b9a4e>:233: RuntimeWarning: overflow encountered in scalar subtract
  (center_x - radius <= 0)


cercles détectés : 28
nb pièces réelles : 3
IMAGE IMG_1652.JPG :
Successfully opened the following image: IMG_1652.JPG
cercles détectés : 4
nb pièces réelles : 4
IMAGE IMG_20240206_213036.jpg :
Successfully opened the following image: IMG_20240206_213036.jpg
cercles détectés : 14
nb pièces réelles : 14
IMAGE IMG_20240206_213214.jpg :
Successfully opened the following image: IMG_20240206_213214.jpg
cercles détectés : 3
nb pièces réelles : 4
IMAGE IMG_20240206_213231.jpg :
Successfully opened the following image: IMG_20240206_213231.jpg
cercles détectés : 3
nb pièces réelles : 4
IMAGE IMG_20240206_213239.jpg :
Successfully opened the following image: IMG_20240206_213239.jpg
cercles détectés : 1
nb pièces réelles : 6
IMAGE IMG_20240206_213252.jpg :
Successfully opened the following image: IMG_20240206_213252.jpg
cercles détectés : 2
nb pièces réelles : 6
IMAGE IMG_5951.HEIC.png :
Could not open or find the image named: IMG_5951.HEIC.png
Successfully opened the following .heic image: IMG_5